# Dr. Kim Morita: Preventive Oral Care Expert, Morita Dentistry

### “Did you know that a healthy mouth can protect you from far more than tooth decay? Diabetes, heart disease, stroke, and a multitude of other diseases and medical conditions have been linked to dental disease. As such, balancing the mouth’s bacterial ecosystem is a great way to stop cavities, stop gum disease, and improve overall health.” —via [Stanford BeWell](https://events.stanford.edu/event/dental_wellness_2617)

### With Dr. Morita, we examine an AI-enhanced patient experience challenge with an automation to use for her morning huddles. 

As Dr. Morita's practice has grown, it becomes more complex and difficult to achieve personalized care at scale. Not only is there a need to know the patient's preferred scheduling for times and days, but also there are often appointment specific preferences like:

* warm water during cleaning
* blankets or pillow
* whether the patient needs premedication
* fluoride/no fluorida
* anesthesia preferences

Dr. Morita's office does morning huddles to help highlight a handful of notable cases for the day's schedule. It can be a lot of information for the team to process the volume of details. How can she simplify the process and yet create highly customized schedules?

## 🔥 Let's get the required packages and fire up a kernel

In [1]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: YamlDotNet, 13.7.1"

Installed Packages Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.3.0-alpha YamlDotNet, 13.7.1

In [2]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## 🧑‍🍳 Let's get cooking on this problem together!

### 🔥 We fire up Semantic Kernel's Experimental Agents ...

In [3]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: YamlDotNet, 13.7.1"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

Installed Packages Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha YamlDotNet, 13.7.1

In [4]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

public class Agenty
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

List<string> agentNames = ["MoritaHuddleManager"];
string team = "";
int i = 1;
List<(string Name, string Instructions)> agentInfo = new();

foreach(var a in agentNames)
{
    var yaml = File.ReadAllText($"../agents/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();

    var p = deserializer.Deserialize<Agenty>(yaml);
    Console.WriteLine($"Agent: {a}");
    team += $"{i}) {p.Name}: {p.Description}, ";
    i++;
    agentInfo.Add((p.Name, p.Instructions));
}

agentInfo

Agent: MoritaHuddleManager


index value 0 (Huddle Manager, Your job is to deliver custom instructions to members of the team at Morita Dentistry each morning. \nSome of the patients have special requirements. You will be informed about the patients with special\nrequirements that will need to be considered during the day, and given simplifi... Item1 Huddle Manager Item2 Your job is to deliver custom instructions to members of the team at Morita Dentistry each morning. 
Some of the patients have special requirements. You will be informed about the patients with special
requirements that will need to be considered during the day, and given simplified instructions
for how to best manage that patient's care. 
The members of the Morita Dentistry team are Pam, Jim, Jan, and Steve.

### 🐣 Let's hatch the agent(s)

In [5]:
#pragma warning disable SKEXP0101

static readonly List<IAgent> s_agents = new();

async Task<IAgent> CreateAgentAsync(string name, string jobDescription)
{
    return Track(
        await new AgentBuilder()
            .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
            .WithInstructions(jobDescription)
            .WithName(name)
            .WithDescription(name)
            .BuildAsync());
}

IAgent Track(IAgent agent)
{
    s_agents.Add(agent);

    return agent;
}

var agents = new List<IAgent>();
foreach (var (name, jobDescription) in agentInfo)
{
    var agent = await CreateAgentAsync(name, jobDescription);

    agents.Add(agent);
}

agents

index value 0 Microsoft.SemanticKernel.Experimental.Agents.Internal.Agent Id asst_HFSzpIWYg0Kwujt3JFn3P5Xy Kernel Microsoft.SemanticKernel.Kernel Plugins [ ] Count 0 (values) (empty) FunctionFilters [ ] Count 0 (values) (empty) PromptFilters [ ] Count 0 (values) (empty) Services Microsoft.Extensions.DependencyInjection.ServiceProvider Culture Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mon, Tue, Wed, Thu, Fri, Sat ] ShortestDayNames [ Su, Mo, Tu, We, Th, Fr, Sa ] DayNames [ Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday ] AbbreviatedMonthNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] IsReadOnly True NativeCalendarName Gregorian Calendar AbbreviatedMonthGenitiveNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthGenitiveNames [ January, February, March, April, May, June, July, August, September, October, November, December, ] Calendar System.Globalization.GregorianCalendar MinSupportedDateTime 1/1/0001 12:00:00 AM MaxSupportedDateTime 12/31/9999 11:59:59 PM AlgorithmType SolarCalendar CalendarType Localized Eras [ 1 ] TwoDigitYearMax 2049 IsReadOnly True OptionalCalendars index value 0 System.Globalization.GregorianCalendar UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant 

### 🧑‍🍳 Let's grab the important info for today

Note that we'll intentionally include a wrong name in the info below to see what happens.

In [6]:
static string patientsToday = @"The special requirements for patients coming today are as follows:

1. Jim's patient at 9AM has a back problem and will need pillows and a blanket
2. Mary's patient at 10AM is a diabetic and will need a blood pressure monitor during the cleaning
";

### 🧵 We need a thread

In [7]:
#pragma warning disable SKEXP0101

IAgentThread? thread = null;

thread = await agents[0].NewThreadAsync();
await thread.AddUserMessageAsync($"{patientsToday}");

### 🕵️💨 And then we're ready to let the agent do its thing

In [8]:
#pragma warning disable SKEXP0101

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

Good morning, team! Here are the special instructions for some of our patients
coming in today:

1. Jim, your 9AM patient has a back problem, so please ensure
that you have pillows and a blanket available to make them comfortable during
the procedure.
   
2. It appears there might have been a mix-up, as we don't
have a team member named Mary listed. However, if Mary was intended to be Pam,
Jan, or Steve, please note that this patient is diabetic. The person treating
this patient will need to have a blood pressure monitor ready to use during the
cleaning for continuous assessment.

Here's a breakdown per team member based on
this information:

- Jim: Set up your room with extra pillows and a blanket
before your 9AM appointment. Make sure the patient's back is well-supported and
they are comfortable throughout their visit.

- Pam/Jan/Steve (The correct
person with the 10AM diabetic patient): Please confirm who has the 10AM
appointment. That team member must have the blood pressure monito

### 🎤 Voice is cool to work with, so let's do a bit of that

In [9]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.34.1"
#r "nuget: NetCoreAudio, 1.8.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0"

#!import ../config/Utils.cs 
#!import ../config/AzureSpeech.cs

Utils.LoadEnvFile();

string subscriptionKey = Environment.GetEnvironmentVariable("AZURE_SPEECH_KEY");
string subscriptionRegion = Environment.GetEnvironmentVariable("AZURE_SPEECH_REGION");

var speechService = new SpeechRecognitionService(subscriptionKey, subscriptionRegion);

Installed Packages Microsoft.CognitiveServices.Speech, 1.34.1 microsoft.extensions.configuration, 8.0.0 Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0 Microsoft.Extensions.Configuration.UserSecrets, 8.0.0 NetCoreAudio, 1.8.0

### 🎤👩🏻 Speak something (for less than 15 seconds and w/o stopping)

In [10]:
string recognizedText = await speechService.RecognizeOnceAsync();
recognizedText

Say something without pausing <up to 15 seconds> ...


Let me say something really interesting.

### 🎤🧑 Let's use this pattern now as the user checking in

In [11]:
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"Transcribed: {recognizedText}");

Say something without pausing <up to 15 seconds> ...
Transcribed: What instructions does Jim need to hear?


### 👂🕵️ The agent can lend their ear

In [12]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync($"{recognizedText}");

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

Good morning, Jim! For your 9AM patient today, please prepare the following:

1.
Set up your treatment room with extra pillows to provide adequate back support
for your patient's comfort due to their back problem.
2. Ensure a blanket is
available in case they need additional support or warmth during the procedure.
Make sure to check in with the patient about their comfort level and adjust the
pillows and blanket as necessary throughout their visit. Have a great day taking
care of your patients!



### 📣🗣️ And we can let the agent speak up

In [15]:
#pragma warning disable SKEXP0101

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content);

Spoke: "Good morning, Jim! For your 9AM patient today, please prepare the following:

1. Set up your treatment room with extra pillows to provide adequate back support for your patient's comfort due to their back problem.
2. Ensure a blanket is available in case they need additional support or warmth during the procedure.

Make sure to check in with the patient about their comfort level and adjust the pillows and blanket as necessary throughout their visit. Have a great day taking care of your patients!"


### 📦 With all the special instructions ready, they can be emailed to all

In [ ]:
#pragma warning disable SKEXP0101

// give the AI a plugin for email here
// and just ask it to send the emails

### 🧼 We'll close with some cleanup

In [22]:
#pragma warning disable SKEXP0101

Console.WriteLine("🧽 Cleaning up ...");

if (thread != null)
{
    Console.WriteLine("Thread going away ...");
    thread.DeleteAsync();
}

if (s_agents.Any())
{
    Console.WriteLine("Agents going away ...");
    // delete all agents
    // loop through the agents and delete them
    foreach (var agent in s_agents)
    {
        await agent.DeleteAsync();
    }
    s_agents.Clear();
}

🧽 Cleaning up ...
Thread going away ...
Agents going away ...
